In [7]:
import requests
import json
from mlProject.utils.ura_token import ura_token
from pandas import json_normalize
import pandas as pd
import psycopg2
import uuid
import pandas.io.sql as psql


In [8]:
from datetime import datetime
from typing import List, Optional
#from pydantic import BaseModel
import json
import psycopg2.extras

# call it in any place of your program
# before working with UUID objects in PostgreSQL
psycopg2.extras.register_uuid()

<psycopg2._psycopg.type 'UUID' at 0x7f849ec1a890>

In [9]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file (if present)
load_dotenv()

# Access environment variables as if they came from the actual environment
ura_access_key = os.getenv('ura_access_key')
ura_access_key

'a02035e2-f6d2-4884-ac12-e44c7463cb98'

In [10]:
u=ura_token(ura_access_key)

In [11]:
df = pd.DataFrame()
for batch in range(1,5):
    url=f"https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch={batch}"
    myobj = {'AccessKey': u.get_accesskey(), 'token': u.get_token(), 'User-Agent': 'Mozilla/5.0'}
    responses = requests.post(url, headers=myobj)

    results = responses.json()['Result']
    flattendata = json_normalize(results,'transaction',['project','street','marketSegment','x','y'],errors='ignore')
    flattendata['contractDate'] = pd.to_datetime(flattendata['contractDate'],format='%m%y')
    df = pd.concat([df,flattendata])
df.head()
    

,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y
0,159.3,-,1,2020-03-01,3,2630000,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.1988965525,30589.1070785135
1,186,-,1,2021-03-01,3,1880000,Strata Terrace,05,Strata,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.1988965525,30589.1070785135
2,257,-,1,2022-05-01,3,4600000,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.1988965525,30589.1070785135
3,524.3,-,1,2020-02-01,3,5500000,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN
4,308,-,1,2021-09-01,3,5200000,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN


In [12]:
df.shape

(132512, 16)

In [13]:
now_str = datetime.now().strftime("%Y%m%d%H%M%S")
df = df.reset_index().rename(columns={'index':'id'})
df['id'] = df['id'].apply(lambda x: str(x)+'_'+now_str)
df.head(2)

,id,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y
0,0_20240904155435,159.3,-,1,2020-03-01,3,2630000,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.1988965525,30589.1070785135
1,1_20240904155435,186,-,1,2021-03-01,3,1880000,Strata Terrace,05,Strata,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.1988965525,30589.1070785135


In [14]:
df.to_csv('transaction.csv')

In [15]:
df['noOfUnits'] = df['noOfUnits'].astype(int)
numerical_cols = ['area','price','nettPrice','x','y']
for col in numerical_cols:
    df[col] = df[col].astype(float)

In [16]:
df.head()

,id,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y
0,0_20240904155435,159.3,-,1,2020-03-01,3,2630000.0,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
1,1_20240904155435,186.0,-,1,2021-03-01,3,1880000.0,Strata Terrace,05,Strata,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
2,2_20240904155435,257.0,-,1,2022-05-01,3,4600000.0,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
3,3_20240904155435,524.3,-,1,2020-02-01,3,5500000.0,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN
4,4_20240904155435,308.0,-,1,2021-09-01,3,5200000.0,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN


In [17]:
df.dtypes

id                       object
area                    float64
floorRange               object
noOfUnits                 int64
contractDate     datetime64[ns]
typeOfSale               object
price                   float64
propertyType             object
district                 object
typeOfArea               object
tenure                   object
nettPrice               float64
project                  object
street                   object
marketSegment            object
x                       float64
y                       float64
dtype: object

In [18]:
from pandantic import BaseModel
from pydantic.types import StrictInt,StrictFloat,StrictStr


In [19]:
class DataFrameSchema(BaseModel):
    area: float
    floorRange: str 
    noOfUnits: StrictInt
    contractDate: Optional[datetime] 
    typeOfSale: str
    price: float
    propertyType: str
    district: str
    typeOfArea: str
    tenure: str
    nettPrice: float
    project: str
    street: str
    marketSegment: str
    x: float
    y: float  




In [20]:
df_valid = DataFrameSchema.parse_df(
    dataframe=df,
    errors="filter",
)

In [21]:
df[df.duplicated(subset=['id'])]

,id,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y
22351,0_20240904155435,106.0,06-10,1,2019-12-01,3,2200000.0,Apartment,10,Strata,Freehold,NaN,RESIDENTIAL APARTMENTS,DRAYCOTT DRIVE,CCR,27709.58839,32445.737240
22352,1_20240904155435,106.0,01-05,1,2021-05-01,3,2400000.0,Apartment,10,Strata,Freehold,NaN,RESIDENTIAL APARTMENTS,DRAYCOTT DRIVE,CCR,27709.58839,32445.737240
22353,2_20240904155435,197.0,01-05,1,2022-01-01,3,4800000.0,Apartment,10,Strata,Freehold,NaN,RESIDENTIAL APARTMENTS,DRAYCOTT DRIVE,CCR,27709.58839,32445.737240
22354,3_20240904155435,140.0,06-10,1,2022-02-01,3,3600000.0,Apartment,10,Strata,Freehold,NaN,RESIDENTIAL APARTMENTS,DRAYCOTT DRIVE,CCR,27709.58839,32445.737240
22355,4_20240904155435,134.0,06-10,1,2023-03-01,3,3400000.0,Apartment,10,Strata,Freehold,NaN,RESIDENTIAL APARTMENTS,DRAYCOTT DRIVE,CCR,27709.58839,32445.737240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132507,25432_20240904155435,208.2,-,1,2024-03-01,3,2600000.0,Semi-detached,28,Land,99 yrs lease commencing from 1997,NaN,GERALD MUGLISTON ESTATE,GERALD CRESCENT,OCR,32836.64710,40981.743260
132508,25433_20240904155435,295.3,-,1,2021-09-01,3,3400000.0,Terrace,27,Land,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR,27055.87048,45402.904304
132509,25434_20240904155435,193.0,01-05,1,2021-05-01,3,1300000.0,Apartment,27,Strata,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR,27055.87048,45402.904304
132510,25435_20240904155435,175.0,-,1,2022-03-01,3,3000000.0,Terrace,27,Land,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR,27055.87048,45402.904304


In [22]:
def save_dataframe_to_postgresql(df: pd.DataFrame, table_name: str, conn_params: dict):
    connection = psycopg2.connect(**conn_params)
    cursor = connection.cursor()
    
    # Create the table if it doesn't exist
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id TEXT PRIMARY KEY,
            area FLOAT,
            floor_range TEXT,
            no_of_units INT,
            contract_date TIMESTAMP,
            type_of_sale TEXT,
            price FLOAT,
            property_type TEXT,
            district TEXT,
            type_of_area TEXT,
            tenure TEXT,
            nett_price FLOAT,
            project TEXT,
            street TEXT,
            market_segment TEXT,
            x FLOAT,
            y FLOAT
        );
    """)
    connection.commit()
    
    # Insert data from DataFrame to PostgreSQL
    for index, row in df.iterrows():
        insert_query = f'''
        INSERT INTO {table_name} (id, area, floor_range, no_of_units, contract_date, type_of_sale, price, property_type, district, type_of_area, tenure, nett_price, project, street, market_segment, x, y)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
        cursor.execute(insert_query, (
            str(uuid.uuid4()), row['area'], row['floorRange'], row['noOfUnits'], row['contractDate'], 
            row['typeOfSale'], row['price'], row['propertyType'], row['district'], row['typeOfArea'], 
            row['tenure'], row['nettPrice'], row['project'], row['street'], row['marketSegment'], 
            row['x'], row['y']
        ))

    # Commit the changes
    connection.commit()
    cursor.close()
    connection.close()



def get_dataframe_from_sql(table_name: str, conn_params: dict):
    connection = psycopg2.connect(**conn_params)
    cursor = connection.cursor()
    df = psql.read_sql(f"Select * from {table_name}", connection)
    return df

In [23]:
# Connect to PostgreSQL
conn_params = {
    "host":"localhost",
    "database":"mydatabase",
    "user":"myuser",
    "password":"mypassword"

}

INGESTION=False
table_name = "propertypricetable"
if INGESTION:
    save_dataframe_to_postgresql(df, table_name,conn_params)

df_fetch = get_dataframe_from_sql(table_name, conn_params)
df_fetch.head()

/var/folders/rj/64nmqpkj7j1dz_vk_9vdrqfw0000gn/T/ipykernel_96405/689271595.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(f"Select * from {table_name}", connection)


,id,area,floor_range,no_of_units,contract_date,type_of_sale,price,property_type,district,type_of_area,tenure,nett_price,project,street,market_segment,x,y
0,bd8e964b-8b01-45ca-9cd5-d5810b9588d4,159.3,-,1,2020-03-01,3,2630000.0,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
1,54d728f7-df61-42fc-ab22-f7ef08bbe7dd,186.0,-,1,2021-03-01,3,1880000.0,Strata Terrace,05,Strata,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
2,b5190668-75e4-49e5-9fc4-81b504d8d35a,257.0,-,1,2022-05-01,3,4600000.0,Terrace,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,NEO PEE TECK LANE,RCR,20914.198897,30589.107079
3,d46fe197-3c71-4278-83ed-51bb2816db8e,524.3,-,1,2020-02-01,3,5500000.0,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN
4,a825db95-5239-4bd9-8e19-f21bb265d42b,308.0,-,1,2021-09-01,3,5200000.0,Semi-detached,05,Land,Freehold,NaN,LANDED HOUSING DEVELOPMENT,ZEHNDER ROAD,RCR,NaN,NaN


connection = psycopg2.connect(**conn_params)
cursor = connection.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)
cursor.close()
connection.close()

In [24]:
df_ec = df.loc[(df['propertyType']=='Executive Condominium')&(df['typeOfSale']=='3')].sort_values(by=['contractDate'])

In [25]:
df_ec.head(2)

,id,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y
126464,19389_20240904155435,120.0,11-15,1,2019-09-01,3,853000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 1997,NaN,YEW MEI GREEN,CHOA CHU KANG NORTH 6,OCR,18819.03588,41984.19449
124149,17074_20240904155435,90.0,11-15,1,2019-09-01,3,875000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604


In [26]:
df_ec.tail(2)

,id,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y
99951,47205_20240904155435,99.0,11-15,1,2024-08-01,3,1780000.0,Executive Condominium,19,Strata,99 yrs lease commencing from 2010,NaN,ESPARINA RESIDENCES,COMPASSVALE BOW,OCR,34460.87998,40428.28628
99769,47023_20240904155435,127.0,11-15,1,2024-08-01,3,1530000.0,Executive Condominium,19,Strata,99 yrs lease commencing from 1997,NaN,THE RIVERVALE,RIVERVALE LINK,OCR,35185.81787,40330.00568


In [27]:
df_ec['noOfUnits'].unique()

array([1])

In [28]:
df_ec['typeOfArea'].unique()

array(['Strata'], dtype=object)

In [29]:
df_ec['tenure'].unique()

array(['99 yrs lease commencing from 1997',
       '99 yrs lease commencing from 2011',
       '99 yrs lease commencing from 2010',
       '99 yrs lease commencing from 1996',
       '99 yrs lease commencing from 2000',
       '99 yrs lease commencing from 2003',
       '99 yrs lease commencing from 2001',
       '99 yrs lease commencing from 2004',
       '99 yrs lease commencing from 2002',
       '99 yrs lease commencing from 2012',
       '99 yrs lease commencing from 2013',
       '99 yrs lease commencing from 2015',
       '99 yrs lease commencing from 2014',
       '99 yrs lease commencing from 2016',
       '99 yrs lease commencing from 2018'], dtype=object)

In [30]:
df_ec['tenure_start'] = df_ec['tenure'].apply(lambda x: int(x.split(' ')[-1]))
print(df_ec['tenure_start'].unique())
df_ec['building_age'] = df_ec['contractDate'].dt.year - df_ec['tenure_start']
df_ec.head()

[1997 2011 2010 1996 2000 2003 2001 2004 2002 2012 2013 2015 2014 2016
 2018]


,id,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,project,street,marketSegment,x,y,tenure_start,building_age
126464,19389_20240904155435,120.0,11-15,1,2019-09-01,3,853000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 1997,NaN,YEW MEI GREEN,CHOA CHU KANG NORTH 6,OCR,18819.03588,41984.19449,1997,22
124149,17074_20240904155435,90.0,11-15,1,2019-09-01,3,875000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604,2011,8
66711,13965_20240904155435,129.0,11-15,1,2019-09-01,3,950000.0,Executive Condominium,18,Strata,99 yrs lease commencing from 1997,NaN,PINEVALE,TAMPINES STREET 73,OCR,39121.60695,37836.26604,1997,22
124155,17080_20240904155435,98.0,06-10,1,2019-09-01,3,825000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604,2011,8
124158,17083_20240904155435,98.0,01-05,1,2019-09-01,3,890000.0,Executive Condominium,23,Strata,99 yrs lease commencing from 2011,NaN,BLOSSOM RESIDENCES,SEGAR ROAD,OCR,20983.51444,41310.75604,2011,8


In [31]:
#